In [55]:
import logging
import pandas as pd
import numpy as np

In [56]:
%load_ext autoreload
%autoreload 2
from IPython.display import display, HTML

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
logging.basicConfig(format='%(asctime)s %(levelname)s %(name)s %(message)s', level=logging.INFO)

In [97]:
import downloader

In [98]:
import download_datasets

In [99]:
dfs=download_datasets.get_datasets()

2022-07-24 19:17:31,792 INFO root processing: data_sources/MaunaLoaCO2MonthlyMean.toml
2022-07-24 19:17:32,707 INFO Downloader Creating dataset MaunaLoaMonthlyCO2
2022-07-24 19:17:32,729 INFO root processing: data_sources/EuropeanMeanTemperatureSinceRomanTime_EuroMed2k.toml
2022-07-24 19:17:33,478 INFO Downloader Creating dataset euromed2k
2022-07-24 19:17:33,502 INFO root processing: data_sources/LawDome2006.toml
2022-07-24 19:17:34,485 INFO Downloader Creating dataset LawDome2006_CO2_CH4_N20
2022-07-24 19:17:34,522 INFO Downloader Creating dataset LawDome2006_CO2_Core_DSS_Eth1996
2022-07-24 19:17:34,525 INFO Downloader Creating dataset LawDome2006_CO2_Core_DSS_MacFM20042006
2022-07-24 19:17:34,528 INFO Downloader Creating dataset LawDome2006_CO2_Core_DE08-2_Eth1996
2022-07-24 19:17:34,530 INFO Downloader Creating dataset LawDome2006_CO2_Core_DE08-2_MacFM20042006
2022-07-24 19:17:34,532 INFO Downloader Creating dataset LawDome2006_CO2_Core_DE08_Eth1996
2022-07-24 19:17:34,535 INFO Dow

In [100]:
print(f" Datasets: {len(dfs.keys())}")
tags=[]
for ds in dfs:
    for tag in dfs[ds]['metadata']['tags']:
        if tag not in tags:
            tags.append(tag)
for tag in tags:
    print(f"----{tag}----:")
    for ds in dfs:
        if tag in dfs[ds]['metadata']['tags']:
            print(f"{ds} ", end="")
    print()
    print()

 Datasets: 40
----co2----:
MaunaLoaMonthlyCO2 LawDome2006_CO2_CH4_N20 LawDome2006_CO2_Core_DSS_Eth1996 LawDome2006_CO2_Core_DSS_MacFM20042006 LawDome2006_CO2_Core_DE08-2_Eth1996 LawDome2006_CO2_Core_DE08-2_MacFM20042006 LawDome2006_CO2_Core_DE08_Eth1996 LawDome2006_CO2_Core_DE08_MacFM20042006 LawDome2006_CO2_Core_DSSW20Kfirn LawDome2006_CO2_Core_CAPEGRIM LawDome2006_CO2_All LawDome2006_CH4_Core_DSS_Eth1996 LawDome2006_CH4_Core_DSS_MacFM20042006 LawDome2006_CH4_Core_DE08-2_Eth1996 LawDome2006_CH4_Core_DE08-2_MacFM20042006 LawDome2006_CH4_Core_DE08_Eth1996 LawDome2006_CH4_Core_DE08_MacFM20042006 LawDome2006_CH4_Core_DSSW20Kfirn LawDome2006_CH4_Core_CAPEGRIM LawDome2006_CH4_All LawDome2006_N20_Core_DSS_MacFM20042006 LawDome2006_N20_Core_DSS08-2_MacFM20042006 LawDome2006_N20_Core_DSS08_MacFM20042006 LawDome2006_N20_Core_DSSW20Kfirn LawDome2006_N20_Core_CAPEGRIM LawDome2006_N20_All MaunaLoaWeeklyHistoricalCO2 LawDome2018CO2 DomeC2016CO2 CenozoicCO2 CenozoicCO2Filtered 

----mauna_loa----:
M

In [101]:
for dataset_name in dfs.keys():
    print()
    print("---------------------------------------------------------------------------------")
    print(f"Dataset {dataset_name} {dfs[dataset_name]['metadata']['tags']}")
    print()
    for entry in dfs[dataset_name]['metadata']:
        if entry in ['title', 'tags']:
            continue
        display(HTML(f"{entry}: {dfs[dataset_name]['metadata'][entry]}"))
    display(dfs[dataset_name]['data'])


---------------------------------------------------------------------------------
Dataset MaunaLoaMonthlyCO2 ['co2', 'mauna_loa', 'direct_measurement']



,year,month,decimal date,average,deseasonalized,ndays,sdev,unc
0,1958,3,1958.2027,315.70,314.43,NaN,NaN,NaN
1,1958,4,1958.2877,317.45,315.16,NaN,NaN,NaN
2,1958,5,1958.3699,317.51,314.71,NaN,NaN,NaN
3,1958,6,1958.4548,317.24,315.14,NaN,NaN,NaN
4,1958,7,1958.5370,315.86,315.18,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
767,2022,2,2022.1250,419.28,418.32,27.0,0.92,0.34
768,2022,3,2022.2083,418.81,417.33,30.0,0.78,0.27
769,2022,4,2022.2917,420.23,417.56,28.0,0.85,0.31
770,2022,5,2022.3750,420.99,417.60,30.0,0.76,0.27



---------------------------------------------------------------------------------
Dataset euromed2k ['temperature', 'tree_ring', 'europe', 'summer']



,Year,Mean,Lower2sigma,Upper2sigma
0,-138,-0.037309,-0.85472,0.76376
1,-137,-0.029138,-0.92172,0.81199
2,-136,-0.066622,-0.91555,0.74954
3,-135,-0.435790,-1.27780,0.35291
4,-134,-0.503160,-1.32540,0.27134
...,...,...,...,...
2136,1999,0.789580,0.13234,1.57860
2137,2000,0.852750,0.21043,1.60250
2138,2001,0.947810,0.27156,1.81780
2139,2002,1.388100,0.73063,2.08610



---------------------------------------------------------------------------------
Dataset LawDome2006_CO2_CH4_N20 ['co2', 'ch4', 'n20', 'icecore', 'antarctica']



...

,Age,Accumulation
0,0.144043,0.244106
1,0.172852,0.246155
2,0.201660,0.248822
3,0.230469,0.249856
4,0.259277,0.249943
...,...,...
1692,48.888200,0.092122
1693,48.917000,0.091968
1694,48.945800,0.091857
1695,48.974600,0.091739



---------------------------------------------------------------------------------
Dataset CenozoicCO2 ['co2']



,proxy,first_author_last_name,publication_year,doi,age_ka,Age_uncertainty_pos_ka,Age_uncertainty_neg_ka,CO2_ppm,CO2_uncertainty_pos_ppm,CO2_uncertainty_neg_ppm,...,cloud,cloud_err,N_cloud_err,LAI,LAI_err,N_LAI_err,attenuation,atten_err,N_atten_err,general notes
0,liverworts,Fletcher,2008,10.1038/ngeo.2007.29 and 10.1111/j.1472-4669.2...,52200,3600.0,3600.0,1967,1534,875,...,0.55,NaN,taken from Fletcher et al. (2008); no error is...,NaN,NaN,taken from Fletcher et al. (2008); the combine...,0.5,NaN,taken from Fletcher et al. (2008); no error is...,Inputs measured but CO2 not reported in Fletch...
1,liverworts,Fletcher,2008,10.1038/ngeo.2007.29,60400,4600.0,4600.0,683,199,201,...,0.55,NaN,taken from Fletcher et al. (2008); no error is...,NaN,NaN,taken from Fletcher et al. (2008); the combine...,0.5,NaN,taken from Fletcher et al. (2008); no error is...,data doi: and 10.1111/j.1472-4669.2012.00320.x



---------------------------------------------------------------------------------
Dataset CenozoicCO2Filtered ['co2']



,age_ka,Age_uncertainty_pos_ka,Age_uncertainty_neg_ka,CO2_ppm,CO2_uncertainty_pos_ppm,CO2_uncertainty_neg_ppm
0,52200,3600.0,3600.0,1967,1534,875
1,60400,4600.0,4600.0,683,199,201


In [ ]:
em2k=dfs['euromed2k']['data'].to_numpy()
gjmt6=dfs['germany_june_meantemp']['data'].to_numpy()
gjmt7=dfs['germany_july_meantemp']['data'].to_numpy()
gjmt8=dfs['germany_august_meantemp']['data'].to_numpy()
sunsp=dfs['sunspots11000yrs']['data'].to_numpy()
sunsp1749=dfs['sunspotsmonthlymean1749']['data'].to_numpy()

In [ ]:
sunsp[:,0]=1950-sunsp[:,0]

In [ ]:
gjmt=(gjmt6[:-1,:]+gjmt7+gjmt8)/3.0

In [ ]:
np.mean(gjmt[:,2])

In [ ]:
def moving_average(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [ ]:
gjmean=np.mean(gjmt[:,2])

In [ ]:
win=10
em2kmat=moving_average(em2k[:,1],2*win+1)
gjmtmat=moving_average(gjmt[:,2]-gjmean,2*win+1)

In [ ]:
sunma=moving_average(sunsp[:,1],2*win+1)
sunman=moving_average(sunsp1749[:,3],20*win+1)

In [ ]:
sunman

In [ ]:
sunman.shape, sunsp1749.shape

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [14,8]  # inch! :-/

In [ ]:
plt.plot(gjmt[:,0],gjmt[:,2]-gjmean)
plt.plot(em2k[:,0],em2k[:,1])
plt.plot(sunsp[-200:,0],sunsp[-200:,1]/60-0.4)
plt.plot(sunsp1749[10*win:-10*win,2],sunman/200)

In [ ]:
# plt.plot(dfnp[:,0],dfnp[:,1])
plt.plot(gjmt[win:-win,0],gjmtmat)
plt.plot(em2k[win:-win,0],em2kmat)
plt.plot(sunsp[-200:,0],sunsp[-200:,1]/80-0.5) # sunma[-200:]/50-0.5)
plt.plot(sunsp1749[10*win:-10*win,2],sunman/200-0.5)

In [ ]:
plt.plot(em2k[-120+win:-win,0],em2kmat[-120+win:-win])
plt.plot(gjmt[win:-win,0],gjmtmat)
plt.plot(sunsp[-20:,0],sunsp[-20:,1]/80-0.5) # sunma[-20:]/50-0.5)
plt.plot(sunsp1749[10*win:-10*win,2],sunman/200-0.5)

In [ ]:
a=-1500
b=-1400
plt.plot(em2k[a:b,0],em2k[a:b,1])
plt.plot(gjmt[a:b,0],gjmtmat[a-win:b-win])
